In [6]:
import zipfile
import os 
import pandas as pd
import shutil

In [46]:
with zipfile.Zipfile("./이름_생년_성별_10000.zip", "r") as f:
    f.extractall('./data/')

AttributeError: module 'zipfile' has no attribute 'Zipfile'

In [ ]:
gender = os.listdir("./data")[0].split("_")[-1][0] 

In [ ]:
file

In [ ]:
male_list = []
female_list = list()

if not os.path.isdir("./남성"):
    os.mkdir("./남성")
if os.path.isdir("./여성")  == False:
    os.mkdir("./여성")
    
for file in os.listdir("./data"):
    gender = file.split("_")[-1][0]
    #if gender == '1' or gender == '3':
    #    print(gender)
    if gender in ['1', '3']:
        #print("남성", end= " ,")
        with open(f"./data/{file}", 'r', encoding='utf-8') as f:
            male_list.append(['남성', f.read().replace(",","")])
        shutil.move(f"./data/{file}", f"./남성/{file}")
    else:
        #print("여성", end=' ,')
        with open(f"./data/{file}", 'r', encoding='utf-8') as f:
            female_list.append(['여성', f.read().replace(",","")])
        shutil.move(f"./data/{file}", f"./여성/{file}")

In [ ]:
male_df = pd.DataFrame(male_list, columns=['성별', '금액'])
female_df = pd.DataFrame(female_list, columns=['성별', '금액'])

In [ ]:
df = pd.concat([male_df, female_df])

In [ ]:
df['금액'] = df['금액'].astype(int)

In [ ]:
df['금액2'] = list(map(int, df['금액']))

In [ ]:
df.groupby(['성별'])['금액'].mean()

In [47]:
df.groupby(['성별'])[['금액']].mean().loc['여성']

NameError: name 'df' is not defined

In [ ]:
df.groupby(['성별'])[['금액']].mean().reset_index().loc[1]

In [ ]:
df.groupby(['성별'])[['금액']].mean()

In [ ]:
df.groupby(['성별'])[['금액']].mean().iloc[1]

In [ ]:
df.groupby(['성별'], as_index=False)[['금액']].mean()

In [ ]:
# agg -> aggregate
# avg -> mean 
df.groupby(['성별'], as_index=False)[['금액']].agg(['mean','median', 'var', 'std', 'count'])

In [7]:
import pickle
with open("./kbo_detail.pkl", "rb") as f:
    kbo = pickle.load(f)

In [ ]:
data = {'df' : df ,
 'kbo' : kbo}

In [2]:
import pickle
with open("./tmp.pkl", "wb") as f:
    pickle.dump(data, f)

NameError: name 'data' is not defined

In [ ]:
kbo['초등'] = kbo['경력'].apply(lambda x : x.split("-")[0])

In [ ]:
kbo.groupby(['초등'])['선수명'].count().sort_values()[::-1].head(10)

In [ ]:
kbo.groupby(['초등'])['선수명'].count().sort_values(ascending=False) 

In [ ]:
kbo.value_counts(['초등'], normalize=True)

In [ ]:
from datetime import date, datetime 

In [ ]:
datetime.strptime("1997년 04월 03일", "%Y년 %m월 %d일")

In [ ]:
kbo['생년월일'] = kbo['생년월일'].apply(lambda x : datetime.strptime(x.strip(), "%Y년 %m월 %d일"))

In [ ]:
kbo['생존일'] = kbo['생년월일'].apply(lambda x : (datetime.now() - x).days)

In [ ]:
kbo.sort_values(by=['생존일', '선수명'], ascending=[False, True]).head(3)

In [ ]:
kbo['나이'] = kbo['생년월일'].apply(lambda x : datetime.now().year - x.year)

In [ ]:
kbo.team.unique()

In [ ]:
kbo.loc[kbo.team == "고양 히어로즈", 'team']  = '키움 히어로즈'

In [ ]:
kbo.groupby(['team'])['나이'].agg(['mean', 'median', 'var'])

In [ ]:
kbo[kbo.team == "삼성 라이온즈"].value_counts("나이").head(5)

In [ ]:
kbo[kbo.team == "키움 히어로즈"].value_counts("나이", normalize=True).head(5).cumsum()

In [ ]:
kbo

In [8]:
import requests 
from bs4 import BeautifulSoup as BS 

r= requests.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")

bs = BS(r.text)

#<form id="CSRFForm" action="/gscvs/ko/store-services/locations" method="post"><input type="hidden" name="CSRFToken" value="406690e8-0340-4931-b372-4b5519577c3c" />
bs.find("input", {'name': "CSRFToken"})

<input name="CSRFToken" type="hidden" value="63c3a773-7f35-4566-a76d-76788617e377"/>

In [9]:
3476 * 5 / 100

173.8

In [10]:
payload = {"pageNum": "1",
"pageSize": "5",
"searchShopName": "",
"searchSido": "",
"searchGugun": "",
"searchDong": "",
"searchType": "",
"searchTypeService": "0",
"searchTypeToto": "0",
"searchTypeCafe25": "0",
"searchTypeInstant": "0",
"searchTypeDrug": "0",
"searchTypeSelf25": "0",
"searchTypePost": "0",
"searchTypeATM": "0",
"searchTypeWithdrawal": "0",
"searchTypeTaxrefund": "0",
"searchTypeSmartAtm": "0",
"searchTypeSelfCookingUtensils": "0",
"searchTypeDeliveryService": "0",
"searchTypeParcelService": "0",
"searchTypePotatoes": "0",
"searchTypeCardiacDefi": "0",
"searchTypeFishShapedBun": "0",}
total = []
with requests.Session() as s:
    r = s.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")
    bs = BS(r.text)
    token = bs.find("input", {'name': "CSRFToken"})['value']
    #print(bs.find("input", {'name': "CSRFToken"}))
    target = f"http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={token}"
    for x in range(1,3):
        if x % 10 == 0: print(x)
        payload['pageNum'] = x 
        r2 = s.post(target, data=payload)
        total.append( pd.DataFrame(eval(r2.json())['results']))

In [4]:
gs25 = pd.concat(total)

NameError: name 'pd' is not defined

In [5]:
with open("./gs25.pkl", "wb") as f:
    pickle.dump(gs25, f)

NameError: name 'pickle' is not defined

In [ ]:
with open("./gs25.pkl", "rb") as f:
    gs25 = pickle.load(f)

In [ ]:
gs25.shopCode.unique().size

In [ ]:
gs25.reset_index(drop=True, inplace=True)

In [ ]:
gs25

In [ ]:
kbo.describe()

# cu

In [11]:
r= requests.get("https://cu.bgfretail.com/store/list.do?category=store#")
sido = [x.text for x in bs.find("select", id="sido") if x.text != '\n' ][1:]

In [12]:
bs = BS(r.text)

In [13]:
sido = [x.text for x in bs.find("select", id="sido") if x.text != '\n' ][1:]

In [146]:
cu_gugunurl = "https://cu.bgfretail.com/store/GugunList.do"

In [147]:
payload = {"pageIndex" : "1",
"listType" : "",
"jumpoCode" : "",
"jumpoLotto" : "",
"jumpoToto" : "",
"jumpoCash" : "",
"jumpoHour" : "",
"jumpoCafe" : "",
"jumpoDelivery" : "",
"jumpoBakery" : "",
"jumpoFry" : "",
"jumpoMultiDevice" : "",
"jumpoPosCash" : "",
"jumpoBattery" : "",
"jumpoAdderss" : "",
"jumpoSido" : "서울특별시",
"jumpoGugun" : "",
"jumpodong" : "",
"user_id" : "",
"sido" : "서울특별시",
"Gugun" : "",
"jumpoName" : "",}

In [142]:
dong_payload= {"pageIndex": "1",
"listType": "",
"jumpoCode": "",
"jumpoLotto": "",
"jumpoToto": "",
"jumpoCash": "",
"jumpoHour": "",
"jumpoCafe": "",
"jumpoDelivery": "",
"jumpoBakery": "",
"jumpoFry": "",
"jumpoMultiDevice": "",
"jumpoPosCash": "",
"jumpoBattery": "",
"jumpoAdderss": "",
"jumpoSido": "서울특별시",
"jumpoGugun": "강남구",
"jumpodong": "",
"user_id": "",
"sido": "서울특별시",
"Gugun": "강남구",
"jumpoName": "",}

In [149]:
r= requests.get("https://cu.bgfretail.com/store/list.do?category=store#")
sido = [x.text for x in bs.find("select", id="sido") if x.text != '\n' ][1:]
for h in range(len(sido)):
    payload["jumpoSido"]=sido[h]
    payload["sido"]=sido[h]


gugun_r = requests.post(cu_gugunurl, data=payload)
gugun=[x['CODE_NAME'] for x in gugun_r.json()['GugunList']]
cu_dong_url  = "https://cu.bgfretail.com/store/DongList.do"
for i in range(len(gugun)):
    dong_payload["jumpoGugun"]=gugun[i]
    dong_payload["Gugun"]=gugun[i]
    dong = [x['CODE_NAME'] for x in requests.post(cu_dong_url, data=dong_payload).json()['GugunList']]
    # print(dong)


In [150]:
sido

['경기도',
 '서울특별시',
 '경상남도',
 '부산광역시',
 '인천광역시',
 '충청남도',
 '경상북도',
 '강원도',
 '충청북도',
 '제주특별자치도',
 '대구광역시',
 '전라북도',
 '전라남도',
 '대전광역시',
 '광주광역시',
 '울산광역시',
 '세종특별자치시',
 '강원특별자치도']

In [116]:
target_payload = {"pageIndex" : "1",
"listType" : "",
"jumpoCode" : "",
"jumpoLotto" : "",
"jumpoToto" : "",
"jumpoCash" : "",
"jumpoHour" : "",
"jumpoCafe" : "",
"jumpoDelivery" : "",
"jumpoBakery" : "",
"jumpoFry" : "",
"jumpoMultiDevice" : "",
"jumpoPosCash" : "",
"jumpoBattery" : "",
"jumpoAdderss" : "",
"jumpoSido" : "서울특별시",
"jumpoGugun" : "강남구",
"jumpodong" : "논현동", #default 논ㄴ현동
"user_id" : "",
"sido" : "서울특별시",
"Gugun" : "강남구",
"jumpoName" : "",}

In [117]:
target_url = "https://cu.bgfretail.com/store/list_Ajax.do"

In [118]:
import warnings
warnings.filterwarnings(action='ignore')
import re
p = re.compile("sevice[0-9]{2} on")

In [119]:
pd.read_html(requests.post(target_url, data=target_payload).text)[0]

,매장명 / 연락처,주소 / 매장 유형 및 서비스
0,강남거평점,서울특별시 강남구 봉은사로 129 거평타운 (논현동) 거평타운
1,강남논현타운점 07077739695,"서울특별시 강남구 강남대로114길 25, (논현동)"
2,논현APEX점 0234437572,"서울특별시 강남구 도산대로 158, 세양에이펙스타워 (논현동)"
3,논현SK점 0230149660,"서울특별시 강남구 학동로 341, (논현동, 에스케이허브블루) 1층 제108호"
4,논현강남점 025436794,서울특별시 강남구 선릉로131길 25 1층


In [25]:
dong

['개포2동',
 '개포4동',
 '개포동',
 '논현1동',
 '논현2동',
 '논현동',
 '대치1동',
 '대치2동',
 '대치4동',
 '대치동',
 '도곡1동',
 '도곡2동',
 '도곡동',
 '삼성1동',
 '삼성2동',
 '삼성동',
 '세곡동',
 '수서동',
 '신사동',
 '압구정동',
 '역삼1동',
 '역삼2동',
 '역삼동',
 '율현동',
 '일원1동',
 '일원2동',
 '일원동',
 '일원본동',
 '자곡동',
 '청담동']

In [74]:
import re
p = re.compile("sevice[0-9]{2} on")

In [163]:
from tqdm import tqdm
count=0;
cu_name_list=[]
cu_add_list=[]
cu_service_list=[]
r= requests.get("https://cu.bgfretail.com/store/list.do?category=store#")
bs = BS(r.text)
sido = [x.text for x in bs.find("select", id="sido") if x.text != '\n' ][1:]
for h in range(len(sido)):
    payload["jumpoSido"]=sido[h]
    payload["sido"]=sido[h]
    gugun_r = requests.post(cu_gugunurl, data=payload)
    gugun=[x['CODE_NAME'] for x in gugun_r.json()['GugunList']]
    cu_dong_url  = "https://cu.bgfretail.com/store/DongList.do"
    for j in tqdm(range(len(gugun))):
        dong_payload["jumpoSido"]=sido[h]
        dong_payload["sido"]=sido[h]
        dong_payload["jumpoGugun"]=gugun[j]
        dong_payload["Gugun"]=gugun[j]
        dong = [x['CODE_NAME'] for x in requests.post(cu_dong_url, data=dong_payload).json()['GugunList']]
        for k in range(len(dong)):
            target_payload["jumpoSido"]=sido[h]
            target_payload["jumpoGugun"]=gugun[j]
            target_payload["jumpodong"]=dong[k]
            target_payload["sido"]=sido[h]
            target_payload["Gugun"]=gugun[j]
    # count=0
            for i in range(1,20):
                target_payload["pageIndex"]=f"{i}"
                target_bs = BS(requests.post(target_url, data=target_payload).text)
                try:
                    
                    rt = target_bs.find("table", {'summary' : '매장 상세 정보'}).findAll("tr")
        # print(dong[k]+"   "+target_payload["pageIndex"]+"   "+str(len(rt)))
                    if len(rt)>=1:
                        for m in range(1,len(rt)):
                            if rt[m].findAll("td")[0].text.strip()!="등록된 게시물이 없습니다.":
                    # print(rt[m].findAll("td")[0].text.strip()+"    "+rt[1].findAll("td")[1].find("address").text)
                                cu_name_list.append(rt[m].findAll("td")[0].text.strip())
                                cu_add_list.append(rt[1].findAll("td")[1].find("address").text)
                                temp_list=[]
                                for x in rt[m].findAll("td")[1].findAll("li", {"class" : re.compile("sevice[0-9]{2} on")}):
                                    temp_list.append(x.find("img")['src'].split("/")[-1].split("_")[-1].replace(".png", ""))
                        # if x.find("img")['src'].split("/")[-1].split("_")[-1].replace(".png", "")!="":
                        # print(x.find("img")['src'].split("/")[-1].split("_")[-1].replace(".png", ""),end=" , ")
                        # count+=1
                    # print("\n")
                    # print(temp_list)
                                cu_service_list.append(temp_list)
                            else:
                                break
                except:
                    pass
                     # pass
                        
                    # print(temp_list,end="    ")
                    # print(dong[k]+"   "+target_payload["pageIndex"])
                        




        # print(cu_service_list)
        # break
        # print(target_payload["jumpoSido"]+" "+target_payload["jumpoGugun"]+" "+target_payload["jumpodong"]+"  "+target_payload["pageIndex"]+"   "+str(len(rt)))
        # if len(rt)>2:
        #     count+=1
        # if i==19:
        #     print(count) #페이지개수
        # try:
            # target_bs = BS(requests.post(target_url, data=target_payload).text)
            # rt = target_bs.find("table", {'summary' : '매장 상세 정보'}).findAll("tr")
            # print(target_url["jumpoSido"]+" "+target_url["jumpoGugun"]+" "+target_url["jumpodong"]+"  "+str(i))
            # print(len(rt)+"   "+dong[k]+ " " + str(i)+ "  " + rt[m].findAll("td")[0].text.strip())
            # if len(rt)==1:
            #     pass
            #     break;
            # else:
            #     for m in range(1,len(rt)):
            #         print(dong[k]+ " " + str(i)+ "  " + rt[m].findAll("td")[0].text.strip())
                    
                
            # print(rt[1]/findAll("td")
            # if rt[1].findAll("td")[0].text.strip()!="등록된 게시물이 없습니다.":
            #     print(rt[1].findAll("td")[0].text.strip())
                # print(dong[k]+"   "+rt[1].findAll("td")[0].text.strip()+"    "+str(i))
            # print(rt[1].findAll("td")[1].find("address").text)
            # for x in rt[1].findAll("td")[1].findAll("li", {"class" : re.compile("sevice[0-9]{2} on")}):
            #     print(x.find("img")['src'].split("/")[-1].split("_")[-1].replace(".png", ""))
        # except:
        #     pass
        
   

 23%|██████████████████▋                                                                | 7/31 [15:05<51:43, 129.30s/it]


ConnectionError: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))

In [137]:
print(len(cu_service_list))

206


In [91]:
print(cu_service_list)

67


In [125]:
target_bs = BS(requests.post(target_url, data=target_payload).text)
rt = target_bs.find("table", {'summary' : '매장 상세 정보'}).findAll("tr")
# print(rt)

In [126]:
rt[0]

<tr> <th scope="col">매장명 / 연락처</th> <th scope="col">주소 / 매장 유형 및 서비스</th> </tr>

In [127]:
len(rt) #개수

6

In [128]:
print(rt[5].findAll("td")[0].text.strip())

논현강남점 025436794


In [129]:
rt[1].findAll("td")[1].find("address").text

' 서울특별시 강남구 봉은사로 129 거평타운 (논현동) 거평타운'

In [130]:
rt[0].findAll("td")[1].findAll("img")[0]

IndexError: list index out of range

In [124]:
rt[1].findAll("td")[1].findAll("img")[0]['src'].split("/")[-1].split("_")[-1].replace(".png", "")

IndexError: list index out of range

In [99]:
import re
p = re.compile("sevice[0-9]{2} on")

In [76]:
print(rt[1])

<tr> <td> <span class="name">강남거평점</span><br/> <span class="tel"></span> </td> <td> <div class="detail_info"> <address> <a href="#" onclick="searchLatLng('서울특별시 강남구 논현동 203-1번지 거평타운', '44389'); return false;">서울특별시 강남구 봉은사로 129 거평타운 (논현동) 거평타운</a></address> <ul> <li class="sevice01 on"><img alt="24시간" src="/images/store/small_01_hour.png"/></li><!-- 서비스되는 li에 class="on" 추가 --> <li class="sevice02 on"><img alt="24시간" src="/images/store/small_02_delivery.png"/></li> <li class="sevice03"><img alt="24시간" src="/images/store/small_03_bakery.png"/></li> <li class="sevice04 on"><img alt="24시간" src="/images/store/small_04_fried.png"/></li> <li class="sevice05 on"><img alt="24시간" src="/images/store/small_05_coffee.png"/></li> <li class="sevice06"><img alt="24시간" src="/images/store/small_06_lotto.png"/></li> <li class="sevice07"><img alt="24시간" src="/images/store/small_07_toto.png"/></li> <li class="sevice08 on"><img alt="24시간" src="/images/store/small_08_cash.png"/></li> <!-- (issues/575)2021.02

In [111]:
test_temp=[]
for i in range(1,6):
    little_temp=[]
    for x in rt[5].findAll("td")[1].findAll("li", {"class" : re.compile("sevice[0-9]{2} on")}):
        # print(x.find("img")['src'].split("/")[-1].split("_")[-1].replace(".png", ""))
        little_temp.append(x.find("img")['src'].split("/")[-1].split("_")[-1].replace(".png", ""))
    test_temp.append(little_temp)    
print(test_temp)

IndexError: list index out of range

In [ ]:
for x in rt[1].findAll("td")[1].findAll("img"):
    print(x['src'].split("/")[-1].split("_")[-1].replace(".png", ""))